In [61]:
# Imports and global variables
import requests
import json

BASE_URL = "http://localhost:8080/engine-rest"

In [65]:
# CLEANUP
# delete all process definitions and running instances
# ! we need to delete all the running instances first

def status_code_successful(status_code: int):
    return str(status_code)[0] == '2'


def delete_all_data(target: str):
    POSSIBLE_TARGETS = ["process-instance", "process-definition", "deployment", "decision-definition"]
    if target not in POSSIBLE_TARGETS:
        raise Exception(str(target) + "not a valid data deletion target")
        
    get_response = requests.get(BASE_URL + "/" + target)
    #print(json.dumps(get_response.json(), indent=2))

    for elem in get_response.json():
        current_id = elem.get('id')
        del_response = requests.delete(BASE_URL + "/" + target + "/" + str(current_id))
        assert(status_code_successful(del_response.status_code))

    
    get_response = requests.get(BASE_URL + "/" + target)
    assert(len(get_response.json()) == 0)   

    
for elem in ["process-instance", "process-definition", "deployment", "decision-definition"]:
    delete_all_data(elem)


In [63]:
# deploy ONE example process instance
# TODO: extent for multiple

multipart_form_data = {
    'deployment-name': (None, 'store'),
    'data': ('testCheckProcess.bpmn', open('testCheckProcess.bpmn', 'r')),
}

response = requests.post(BASE_URL + "/deployment/create", files=multipart_form_data)

#print(json.dumps(response.json(), indent=2))
for elem in response.json().get('deployedProcessDefinitions'):
    new_process_id = elem

#print(new_process_id)

In [64]:
# start instance

headers = {'Content-Type': 'application/json'}
response = requests.post("http://localhost:8080/engine-rest/process-definition/" + str(new_process_id)+ "/start", headers=headers)
#print(json.dumps(response.json(), indent=2))
assert(status_code_successful(response.status_code))


In [ ]:
# TODO
# next steps:
# create automatically running bp
# start it from here
# poll history service about it (to calculate rewards)